In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [44]:
from functools import partial
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from bdr import Features
from bdr.data.imdb_faces import load_faces

In [3]:
import sys
sys.path.append('../experiments/')
import train_test

In [4]:
full = load_faces(load_their_probs=True)

In [5]:
full

<Features: 19,645 bags with 1 to 796 4083-dimensional points (397,949 total)>

In [35]:
_split = {}
def split(seed, feats=full):
    if feats is full and seed in _split:
        r = _split[seed]
        # have we added any metadata since splitting?
        if set(r[0].meta) == set(full.meta):
            return _split[seed]
    
    parser = train_test.make_parser()
    args = parser.parse_args(
        ['imdb_faces', '--type=fourier', 'fake_out', '--split-seed={}'.format(seed)])
    spl = train_test._split_feats(args, feats)
    
    if feats is full:
        _split[seed] = spl
    return spl

In [12]:
seeds = np.arange(20, 30)

### Their model

#### Method: posterior averaging

In [47]:
_labels = np.arange(101)

def cdf_vals(discrete_preds, y):
    pred_cdfs = np.cumsum(discrete_preds, axis=1)
    
    lo = y.astype(int)
    portion = y % 1
    lo_cdf = pred_cdfs[range(len(y)), lo]
    hi_cdf = pred_cdfs[range(len(y)), lo + 1]
    return lo_cdf + (hi_cdf - lo_cdf) * portion

def _do_posterior(posteriors, y, d, name):
    s = partial('{}_{}'.format, name)
    d[s('y')] = y
    d[s('preds')] = preds = posteriors.dot(_labels)
    d[s('pred_vars')] = posteriors.dot(_labels ** 2) - preds ** 2
    d[s('pred_nlls')] = pred_nlls = -np.log(posteriors[range(len(y)), y.round().astype(int)]).mean()
    d[s('pred_cdfs')] = pred_cdfs = cdf_vals(posteriors, y)
    d[s('mse')] = mean_squared_error(y, preds)
    d[s('r2')] = r2_score(y, preds)
    d[s('nll')] = pred_nlls.mean()
    d[s('coverage')] = ((0.025 < pred_cdfs) & (pred_cdfs < 0.975)).mean()

In [48]:
!mkdir -p theirs_avg
for seed in seeds:
    train, estop, val, test = split(seed)
    d = {}
    for name, ds in [('val', val), ('test', test)]:
        posteriors = np.vstack([np.mean(x, axis=0) for x in ds.probs])
        _do_posterior(posteriors, ds.y, d, name)    
    np.savez('theirs_avg/seed_{}.npz'.format(seed), **d)